# First, let us set up a few dependencies

Don't forget to switch to a GPU-enabled colab runtime!

```
Runtime -> Change Runtime Type -> GPU
```

In [1]:
import os
import contextlib
@contextlib.contextmanager
def directory(name):
  ret = os.getcwd()
  os.chdir(name)
  yield None
  os.chdir(ret)

import subprocess
def run(input, exception_on_failure=False):
  try:
    program_output = subprocess.check_output(f"{input}", shell=True, universal_newlines=True, stderr=subprocess.STDOUT)
  except Exception as e:
    if exception_on_failure:
      raise e
    program_output = e.output

    return program_output
def prun(input, exception_on_failure=False):
  x = run(input, exception_on_failure)
  print(x)
  return x

# This mounts your google drive to this notebook. You might have to change the path to fit with your dataset folder inside your drive.

Read the instruction output by the cell bellow carefully!

In [2]:
# Create a temporary workspace
import tempfile


SESSION_WORKSPACE = tempfile.mkdtemp()
print(f"Session workspace created at: {SESSION_WORKSPACE}")

Session workspace created at: /tmp/tmpesliwgsd


In [3]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')
DRIVE_PATH = "/content/drive/My Drive"

Mounted at /content/drive


In [4]:
# Unzip the dataset
import shutil
import os


DATASET_DIR_NAME = "duckietown_object_detection_dataset"
DATASET_ZIP_NAME = f"{DATASET_DIR_NAME}.zip"
DATASET_DIR_PATH = os.path.join(SESSION_WORKSPACE, DATASET_DIR_NAME)
TRAIN_DIR = "train"
VALIDATION_DIR = "val"
IMAGES_DIR = "images"
LABELS_DIR = "labels"


def show_info(base_path: str):
  for l1 in [TRAIN_DIR, VALIDATION_DIR]:
    for l2 in [IMAGES_DIR, LABELS_DIR]:
      p = os.path.join(base_path, l1, l2)
      print(f"#Files in {l1}/{l2}: {len(os.listdir(p))}")


def unzip_dataset():
  # check zipped file
  zip_path = os.path.join(DRIVE_PATH, DATASET_ZIP_NAME)
  assert os.path.exists(zip_path), f"No zipped dataset found at {zip_path}! Abort!"

  # unzip the data
  print("Unpacking zipped data...")
  shutil.unpack_archive(zip_path, DATASET_DIR_PATH)
  print(f"Zipped dataset unpacked to {DATASET_DIR_PATH}")

  # show some info
  show_info(DATASET_DIR_PATH)


unzip_dataset()

Unpacking zipped data...
Zipped dataset unpacked to /tmp/tmpesliwgsd/duckietown_object_detection_dataset
#Files in train/images: 804
#Files in train/labels: 804
#Files in val/images: 202
#Files in val/labels: 202


In [5]:
# change  working directory to the session workspace
os.chdir(SESSION_WORKSPACE)
print(f"PWD: {os.getcwd()}")

# install pytorch and torchvision
!pip3 install torch==1.13.0 torchvision==0.14.0

PWD: /tmp/tmpesliwgsd
ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.15.0
ERROR: Could not find a version that satisfies the requirement torchvision==0.14.0 (from versions: 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.16.2, 0.17.0, 0.17.1, 0.17.2, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 0.20.0, 0.20.1, 0.21.0, 0.22.0)
ERROR: No matching distribution found for torchvision==0.14.0


# Next, we will clone Yolov5

In [15]:
!rm -rf ./yolov5
!git clone https://github.com/ultralytics/yolov5.git -b v7.0
!cd yolov5 && pip3 install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17413, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 17413 (delta 64), reused 22 (delta 22), pack-reused 17327 (from 4)
Receiving objects: 100% (17413/17413), 16.30 MiB | 20.31 MiB/s, done.
Resolving deltas: 100% (11935/11935), done.
Note: switching to '915bbf294bb74c859f0b41f1c23bc395014ea679'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



# We now inform the training process of the format and location of our dataset

In [14]:
%%writefile ./yolov5/data/duckietown.yaml

# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ../duckietown_object_detection_dataset/train
val: ../duckietown_object_detection_dataset/val

# number of classes
nc: 4

# class names
names: [ 'duckie', 'cone', 'truck', 'bus' ]

Overwriting ./yolov5/data/duckietown.yaml


# And we're ready to train! This step will take about 5 minutes.

Notice that we're only training for 10 epochs. That's probably not enough!

In [16]:
!cd yolov5 && python3 train.py --cfg ./models/yolov5n.yaml --img 416 --batch 32 --epochs 10 --data duckietown.yaml --weights yolov5n.pt

2025-05-07 09:08:02.588394: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746608882.608459    4228 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746608882.614937    4228 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-07 09:08:02.636902: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: E

In [9]:
import numpy as np

all_exps = os.listdir("yolov5/runs/train")
all_exps_filtered = map(lambda x: int(x.replace("exp", "1")), filter(lambda x: x.startswith("exp"), all_exps))
all_exps_filtered = np.array(list(all_exps))
latest_exp_index = np.argmax(all_exps)
latest_exp = all_exps[latest_exp_index]
print(f"Latest exp is {latest_exp}")

prun(f"cp yolov5/runs/train/{latest_exp}/weights/best.pt yolov5/best.pt")
print(f"Marked the model from the latest run ({latest_exp}) as yolov5/best.pt.")

Latest exp is exp
cp: cannot stat 'yolov5/runs/train/exp/weights/best.pt': No such file or directory

Marked the model from the latest run (exp) as yolov5/best.pt.


# Next, we can upload your model to Duckietown's cloud!

We will need our token to access our personal cloud space.

In [11]:
# TODO: Fill in the duckietown token here
YOUR_DT_TOKEN = "dt1-HeFLq6vDkJmSLHH1txvYgYLYBVWhq2NPkpT6k-43dzqWFnWd8KBa1yev1g3UKnzVxZkkTbfaVLa8CvtvbYodXK1yuNcFH4VEgrERoS7R"

Then, we chose the location of the trained model on disk and its name once uploaded to our cloud space. You should not change these values, or the robots will not be able to find the model to download.

In [12]:
import sys
sys.path.insert(0, './yolov5')

# DO NOT CHANGE THESE
model_name = "yolov5n"
model_local_path = "./yolov5/best.pt"
model_remote_path = f"courses/mooc/objdet/data/nn_models/{model_name}.pt"

# install DCSS client
!pip3 install dt-data-api

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 9.6 MB/s eta 0:00:00
  Created wheel for dt-data-api: filename=dt_data_api-2.1.0-py3-none-any.whl size=14363 sha256=4a3a0df5a64fdfcf8327acdb84296f7d8f24e2c41a853f9b41e63ec0beaddca9
  Stored in directory: /root/.cache/pip/wheels/f9/b5/60/d35f7f09040a1b33db975be4f28603b344bf1e9d2bddef6ac7
  Created wheel for dt-authentication: filename=dt_authentication-2.2.0-py3-none-any.whl size=10095 sha256=de1c21b8ff50215f38173b0e45980de76ef70428a1cd578fab541614e0bb03aa
  Stored in directory: /root/.cache/pip/wheels/f7/d5/e7/985e33386d61f6b4027fe7f9a2afec5a491972b3948b71a425
Successfully built dt-data-api dt-authentication


We now open a pointer to our cloud space and upload the model.

In [13]:
import torch
from dt_data_api import DataClient, Storage

# open a pointer to our personal duckietown cloud space
client = DataClient(YOUR_DT_TOKEN)
storage = client.storage("user")

# upload model
upload = storage.upload(model_local_path, model_remote_path)
upload.join()

ValueError: The file /tmp/tmpesliwgsd/yolov5/best.pt does not exist.

# Done!

We're done training! You can now close this tab and go back to the `Training` notebook